<H1>1) Read Data </H1>




1.1 Spark Natively reads the CSV file into Dataframe . Create Weather_org_df ( Weather original Dataframe)

In [2]:
import  org.apache.spark._ 
val weather_org_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("weatherAUS.csv")

import org.apache.spark._
weather_org_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


1.2 Check the Schema of the Data frame

In [2]:
weather_org_df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: string (nullable = true)
 |-- MaxTemp: string (nullable = true)
 |-- Rainfall: string (nullable = true)
 |-- Evaporation: string (nullable = true)
 |-- Sunshine: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: string (nullable = true)
 |-- Humidity9am: string (nullable = true)
 |-- Humidity3pm: string (nullable = true)
 |-- Pressure9am: string (nullable = true)
 |-- Pressure3pm: string (nullable = true)
 |-- Cloud9am: string (nullable = true)
 |-- Cloud3pm: string (nullable = true)
 |-- Temp9am: string (nullable = true)
 |-- Temp3pm: string (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RISK_MM: double (nullable = true)
 |-- RainTomorrow: string (nullable = true)



1.3  Create temp table Aus_Rain_Data called for easy data manipulation operations 

In [3]:
weather_org_df.registerTempTable("Aus_Rain_Data")

1.4 Try Reading some data

In [1]:
spark.sql("select * from Aus_Rain_Data").show(5)

Intitializing Scala interpreter ...

Spark Web UI available at http://9c12c0ba0efc:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1590298426800)
SparkSession available as 'spark'


2020-05-24 05:33:44,299 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


org.apache.spark.sql.AnalysisException:  Table or view not found: Aus_Rain_Data; line 1 pos 14

<H1>2) Data Analysis  </H1>

2.1 How many locations the weather is recorded to predict rain the different zone

In [ ]:
weather_org_df.select(mean(weather_org_df("MaxTemp"))).show()

In [7]:
weather_org_df.select("MinTemp").filter(weather_org_df("MinTemp") < 0 AND weather_org_df("Location") = Albury) .show()

<console>: 31: error: value AND is not a member of org.apache.spark.sql.Column

In [8]:
spark.sql("SELECT count(distinct location) from Aus_Rain_Data ").show(100)

+------------------------+
|count(DISTINCT location)|
+------------------------+
|                      49|
+------------------------+



Data contains weather data collected from 49 Stations 

In [9]:
 spark.sql("select MinTemp  from Aus_Rain_Data where Date between '2008-12-01' and '2009-01-01' and Location = 'Albury' ").show()
spark.sql("select count(MinTemp) as missingValues  from Aus_Rain_Data where MinTemp = 'NA' ").show()

+-------+
|MinTemp|
+-------+
|   13.4|
|    7.4|
|   12.9|
|    9.2|
|   17.5|
|   14.6|
|   14.3|
|    7.7|
|    9.7|
|   13.1|
|   13.4|
|   15.9|
|   15.9|
|   12.6|
|    9.8|
|   14.1|
|   13.5|
|   11.2|
|    9.8|
|   11.5|
+-------+
only showing top 20 rows

+-------------+
|missingValues|
+-------------+
|          637|
+-------------+



In [10]:
spark.sql("select Location, AVG(MinTemp)  from Aus_Rain_Data  where location = 'Albury' group by Location, MONTH(date) ").show(50)

+--------+----------------------------+
|Location|avg(CAST(MinTemp AS DOUBLE))|
+--------+----------------------------+
|  Albury|           14.54331983805668|
|  Albury|            5.83203463203463|
|  Albury|           9.502083333333333|
|  Albury|          12.552916666666665|
|  Albury|          3.6544354838709703|
|  Albury|           5.455434782608697|
|  Albury|           16.74657534246576|
|  Albury|           3.435984848484848|
|  Albury|           8.321810699588474|
|  Albury|          16.874193548387098|
|  Albury|          13.721691176470591|
|  Albury|          3.7752032520325205|
+--------+----------------------------+



In [11]:
val p1 = "('1','2','3','4','5','6','7','8','9','10','11','12')"
val p2 = "('Albury','Sydney')"

val query  = s"select Location,AVG(MinTemp) from Aus_Rain_Data where Location in $p2   group by Location"

spark.sql(query).show()

+--------+----------------------------+
|Location|avg(CAST(MinTemp AS DOUBLE))|
+--------+----------------------------+
|  Sydney|          14.865056988602275|
|  Albury|            9.52089850249585|
+--------+----------------------------+



p1: String = ('1','2','3','4','5','6','7','8','9','10','11','12')
p2: String = ('Albury','Sydney')
query: String = select Location,AVG(MinTemp) from Aus_Rain_Data where Location in ('Albury','Sydney')   group by Location


In [6]:
weather_org_df = weather_org_df.withColumn("MinTemp", F.when(F.col("MinTemp")=='NA', 0).otherwise(F.col("MinTemp")))

<console>: 2: error: unclosed character literal

In [27]:

weather_org_df.withColumn("MinTempA", when($"MinTemp"= NA, weather_org_df.select(mean("MinTemp"))
  .first()(0).asInstanceOf[Double])
  .otherwise($"MinTemp"))



<console>: 31: error: missing argument list for method $ in class StringToColumn

In [29]:
weather_org_df.registerTempTable("Test")

In [30]:
spark.sql("select MinTemp from Test where Mintemp = 'NA' ").show()

+-------+
|MinTemp|
+-------+
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
+-------+
only showing top 20 rows



In [31]:
weather_org_df.na.fill(weather_org_df.select(mean("MinTemp")),Seq("MinTemp"))

<console>: 31: error: overloaded method value fill with alternatives:

In [34]:

weather_org_df.withColumn("MinTempA", when($"MinTemp".isnull, weather_org_df.select(mean("MinTemp"))
  .first()(0).asInstanceOf[Double])
  .otherwise($"MinTemp"))

<console>: 31: error: value isnan is not a member of org.apache.spark.sql.ColumnName

In [37]:
weather_org_df.NA.fill(0,Seq("MinTemp"))

<console>: 31: error: value NA is not a member of org.apache.spark.sql.DataFrame

In [36]:
spark.sql("select MinTemp from Test where Mintemp = 'NA' ").show()

+-------+
|MinTemp|
+-------+
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
|     NA|
+-------+
only showing top 20 rows

